# Fundamental Analysis
This notebook plots various fundamental statistics of different stocks.  
These links were helpful:  
https://simply-python.com/2015/02/13/getting-historical-financial-statistics-of-stock-using-python/  
http://stackoverflow.com/questions/40139537/scrape-yahoo-finance-financial-ratios  
https://automatetheboringstuff.com/chapter11/  

## Todo
- Just need to fill missing values somehow and then you're good to go. Maybe see if previous value is available, and if so, take that? If previous value is not available, then just skip this stock. But then, need to have desired machine learning columns ahead of time.
- Also need to make sure you're using stocks from the **same sector**, as different sectors can have different debt/equity standards, profit margins, etc.
- Then, just get that ML algorithm working baybee.

In [196]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint as pp
import matplotlib.pyplot as plt
from pandas_datareader.data import DataReader
from pandas.tseries.offsets import BMonthEnd
from pattern.web import URL
%matplotlib inline

In [2]:
tickers = pd.read_csv('csv/s&p500.csv',delimiter=",")
print tickers["Sector"].unique()
tickers.head()

['Industrials' 'Health Care' 'Information Technology'
 'Consumer Discretionary' 'Utilities' 'Financials' 'Materials'
 'Consumer Staples' 'Energy' 'Telecommunications Services']


,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,ABT,Abbott Laboratories,Health Care
2,ABBV,AbbVie,Health Care
3,ACN,Accenture plc,Information Technology
4,ATVI,Activision Blizzard,Information Technology


In [229]:
#scrapes financial data from Morningstar
def get_stock_financials(stock,year):
    path = "Fundamental_Analysis/"
    skiprows = [19,20,31,41,42,43,48,58,53,64,65,72,73,95,101,102]
    del_columns = ['Year over Year','3-Year Average','5-Year Average','10-Year Average','Taxes Payable','Long-Term Debt']
    try:
        d = pd.read_csv('%s%s_keyratios.csv'%(path, stock), header=2, index_col=0, skiprows = skiprows)
    except:
        webpage = "http://financials.morningstar.com/ajax/exportKR2CSV.html?t=%s&culture=en-CA&region=USA&order=asc&r=314562"%stock
        url = URL(webpage)
        f = open('%s%s_keyratios.csv'%(path, stock), 'wb') # save as test.gif
        f.write(url.download())
        f.close()
        d = pd.read_csv('%s%s_keyratios.csv'%(path, stock), header=2, index_col=0, skiprows = skiprows)
    d = d.transpose().reset_index()
    d = d.dropna(axis='columns', how='all')
    d.insert(0,'stock',stock)
    for dc in del_columns: #delete annoying repeated columns
        try:
            del d[dc]
        except:
            pass
    return d[d['index'].str.contains("%d"%year)]

def last_weekday_of_month(date):
    return pd.date_range(date, periods=1, freq='BM')

def get_frac_price_change(stock,date):
    last_current = last_weekday_of_month(date)
    ds = date.split('-')
    last_future = last_weekday_of_month('%d-%s'%(int(ds[0])+1, ds[1]))
    current_price = DataReader(stock, 'yahoo', last_current, last_current)['Adj Close'].values[0]
    future_price = DataReader(stock, 'yahoo', last_future, last_future)['Adj Close'].values[0]
    return (future_price - current_price)/current_price

## Get Data
Process data, could take a moment.

In [281]:
stocks = pd.read_csv('csv/s&p500.csv',delimiter=",")
year = 2014

#get X data and y data
dfcols = get_stock_financials(stocks["Symbol"][0],year).columns
X = pd.DataFrame(columns=columns)
y = []
for s in stocks["Symbol"][:30]:
    try:
        _X = get_stock_financials(s,year)
        _y = get_frac_price_change(s,_X['index'].values[0])
        if _X.shape[1] > 1 and _y:
            X = X.append(_X)
            y.append(_y)
    except:
        print "could not process stock %s"%s
X = X.set_index('stock')

could not process stock AA


In [254]:
X.columns

Index([u'Accounts Payable', u'Accounts Receivable', u'Accrued Liabilities',
       u'Asset Turnover', u'Asset Turnover (Average)',
       u'Book Value Per Share * USD', u'COGS', u'Cap Ex as a % of Sales',
       u'Cap Spending USD Mil', u'Cash & Short-Term Investments',
       u'Cash Conversion Cycle', u'Current Ratio', u'Days Inventory',
       u'Days Sales Outstanding', u'Debt/Equity', u'Dividends USD',
       u'EBT Margin', u'Earnings Per Share USD', u'Financial Leverage',
       u'Financial Leverage (Average)', u'Fixed Assets Turnover',
       u'Free Cash Flow Growth % YOY', u'Free Cash Flow Per Share * USD',
       u'Free Cash Flow USD Mil', u'Free Cash Flow/Net Income',
       u'Free Cash Flow/Sales %', u'Gross Margin', u'Gross Margin %',
       u'Intangibles', u'Interest Coverage', u'Inventory',
       u'Inventory Turnover', u'Net Income USD Mil', u'Net Int Inc & Other',
       u'Net Margin %', u'Net PP&E', u'Operating Cash Flow Growth % YOY',
       u'Operating Cash Flow USD Mi

Grab columns of interest to be used for machine learning.

In [283]:
percentiles = ['Net Margin %','Gross Margin %','Operating Margin %','Return on Assets %', 'Payout Ratio % *',
               'Return on Invested Capital %','Return on Equity %','Free Cash Flow/Sales %']
ratios = ['Book Value Per Share * USD', 'Earnings Per Share USD','Free Cash Flow Per Share * USD', 'Debt/Equity',
         'Current Ratio','Dividends USD','EBT Margin','Quick Ratio']

columns = percentiles + ratios

In [285]:
XX = X[columns]
XX.head()

,Net Margin %,Gross Margin %,Operating Margin %,Return on Assets %,Payout Ratio % *,Return on Invested Capital %,Return on Equity %,Free Cash Flow/Sales %,Book Value Per Share * USD,Earnings Per Share USD,Free Cash Flow Per Share * USD,Debt/Equity,Current Ratio,Dividends USD,EBT Margin,Quick Ratio
stock,,,,,,,,,,,,,,,,
MMM,15.57,48.3,22.4,15.29,43.9,23.10,32.38,16.13,25.46,7.49,7.30,0.52,1.96,3.42,22.08,1.13
ABT,11.28,54.5,12.8,5.42,63.5,7.64,9.78,12.83,15.18,1.49,1.93,0.16,1.45,0.88,12.44,0.76
ABBV,8.89,77.8,17.1,6.25,71.3,11.49,56.91,14.71,2.92,1.10,2.96,6.06,1.41,1.66,11.87,1.11
ACN,9.23,30.4,13.5,16.91,41.8,54.58,55.02,9.93,8.53,4.52,4.00,NaN,1.46,1.86,13.48,1.30
ATVI,18.94,65.4,26.8,5.81,23.3,7.30,12.05,26.88,9.69,1.13,1.19,0.60,2.55,0.20,22.25,2.03


# Extra

In [246]:
s = "AA"
data = get_stock_financials(s,2014)
start = data['index'].values[0]
get_frac_price_change(s,start)
#a= get_stock_price_change("ACN",start)
#print a

RemoteDataError: Unable to read URL: http://ichart.finance.yahoo.com/table.csv

In [88]:
data = get_stock_financials("ACN",2015)
data

d1 = get_stock_financials(stocks["Symbol"][2],2012)

for d in data.columns:
    if d not in d1.columns:
        print d

11
11
11
11


In [190]:
data['index'].values[0]

'2014-08'

In [24]:
stock = "AEM"
webpage = "http://financials.morningstar.com/ajax/exportKR2CSV.html?t=%s&culture=en-CA&region=USA&order=asc&r=314562"%stock
url = URL(webpage)
f = open('Fundamental_Analysis/%s_keyratios.csv'%stock, 'wb') # save as test.gif
f.write(url.download())
f.close()